In [2]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from modules import extract_, compress_, load_
import pandas as pd

In [3]:
spark = SparkSession.builder.getOrCreate()

In [22]:
def transform_flat(date):
    '''
    날짜를 입력하면 그 날짜의 모든 거래를 품목별 법인별로 집계하여\n
    단일 json 데이터로 반환하는 함수
    '''
    import os
    import math
    import json
    import pandas as pd
    from dotenv import load_dotenv
    load_dotenv()

    api_id = os.getenv('garak_id')
    api_pw = os.getenv('garak_passwd')
    url = 'http://www.garak.co.kr/publicdata/dataOpen.do?'

    bubin_list = ['11000101','11000102','11000103','11000104','11000105','11000106']
    pummok_list = ['감귤','감자','건고추','고구마','단감','당근','딸기','마늘','무',
                    '미나리','바나나','배','배추','버섯','사과','상추','생고추','수박',
                    '시금치','양배추','양상추','양파','오이','참외','토마토','파',
                    '포도','피망','호박']

    dict1 = {'data': []}
    for pummok in pummok_list:
        dict2 = {f'{pummok}': []}
        for bubin in bubin_list:
            params = (
                    ('id', api_id),
                    ('passwd', api_pw),
                    ('dataid', 'data12'),
                    ('pagesize', '10'),
                    ('pageidx', '1'),
                    ('portal.templet', 'false'),
                    ('s_date', date),
                    ('s_bubin', bubin),
                    ('s_pummok', pummok),
                    ('s_sangi', '')
                    )
            dict3 = {f'{bubin}': []}
            list_total_count = int(extract_.extract(url, params)['lists']['list_total_count'])
            total_page = math.ceil(int(list_total_count) / 10)

            if int(list_total_count) != 0:
                for page in range(1, total_page+1):
                    params = (
                                ('id', api_id),
                                ('passwd', api_pw),
                                ('dataid', 'data12'),
                                ('pagesize', '10'),
                                ('pageidx', page),
                                ('portal.templet', 'false'),
                                ('s_date', date),
                                ('s_bubin', bubin),
                                ('s_pummok', pummok),
                                ('s_sangi', '')
                             )
                    html_dict = extract_.extract(url, params)
                    if list_total_count % 10 > 1:
                        for i in range(len(html_dict['lists']['list'])):
                            dict3[f'{bubin}'].append({
                                'idx' : ((page -1) * 10) + (i + 1),
                                'PUMMOK' : html_dict['lists']['list'][i]['PUMMOK'],
                                'PUMJONG' : html_dict['lists']['list'][i]['PUMJONG'],
                                'UUN' : html_dict['lists']['list'][i]['UUN'],
                                'DDD' : html_dict['lists']['list'][i]['DDD'],
                                'PPRICE' : html_dict['lists']['list'][i]['PPRICE'],
                                'SSANGI' : html_dict['lists']['list'][i]['SSANGI'],
                                'CORP_NM' : html_dict['lists']['list'][i]['CORP_NM'],
                                'ADJ_DT' : html_dict['lists']['list'][i]['ADJ_DT']
                                })
                    elif list_total_count % 10 == 1:
                        if list_total_count > 1:
                            for i in range(10):
                                dict3[f'{bubin}'].append({
                                    'idx' : ((page -1) * 10) + (i + 1),
                                    'PUMMOK' : html_dict['lists']['list'][i]['PUMMOK'],
                                    'PUMJONG' : html_dict['lists']['list'][i]['PUMJONG'],
                                    'UUN' : html_dict['lists']['list'][i]['UUN'],
                                    'DDD' : html_dict['lists']['list'][i]['DDD'],
                                    'PPRICE' : html_dict['lists']['list'][i]['PPRICE'],
                                    'SSANGI' : html_dict['lists']['list'][i]['SSANGI'],
                                    'CORP_NM' : html_dict['lists']['list'][i]['CORP_NM'],
                                    'ADJ_DT' : html_dict['lists']['list'][i]['ADJ_DT']
                                    })
                            list_total_count -= 10
                        elif list_total_count == 1:
                            dict3[f'{bubin}'].append({
                                'idx' : int(html_dict['lists']['list_total_count']),
                                'PUMMOK' : html_dict['lists']['list']['PUMMOK'],
                                'PUMJONG' : html_dict['lists']['list']['PUMJONG'],
                                'UUN' : html_dict['lists']['list']['UUN'],
                                'DDD' : html_dict['lists']['list']['DDD'],
                                'PPRICE' : html_dict['lists']['list']['PPRICE'],
                                'SSANGI' : html_dict['lists']['list']['SSANGI'],
                                'CORP_NM' : html_dict['lists']['list']['CORP_NM'],
                                'ADJ_DT' : html_dict['lists']['list']['ADJ_DT']
                                })
                dict2[f'{pummok}'].append(dict3)
            else:
                pass
        dict1['data'].append(dict2)

    flattened_data = []
    for item_data in dict1['data']:
        for item, bubin_list in item_data.items():
            for bubin_data in bubin_list:
                for bubin, transactions in bubin_data.items():
                    for transaction in transactions:
                        flattened_row = transaction.copy()
                        flattened_row['item'] = item
                        flattened_row['bubin'] = bubin
                        flattened_data.append(flattened_row)

    return flattened_data

In [3]:
test = {'col1': [1, 2, 3], 'col2': [4, 5, 6]}
test_df = pd.DataFrame(test)

In [4]:
test_df

,col1,col2
0,1,4
1,2,5
2,3,6


In [28]:
tf = spark.createDataFrame(test_df)

In [29]:
tf.show()

Py4JJavaError: An error occurred while calling o171.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChe

In [8]:
data = transform_flat('20230407')

In [9]:
data[0]

{'idx': 1,
 'PUMMOK': '감귤',
 'PUMJONG': '감귤 기타',
 'UUN': '1.5kg',
 'DDD': '특(1등)',
 'PPRICE': '17000',
 'SSANGI': '제주자치도 제주시',
 'CORP_NM': '서울청과',
 'ADJ_DT': '20230407',
 'item': '감귤',
 'bubin': '11000101'}

In [18]:
data = [("2023-04-12",), ("2023-04-13",)]
schema2 = StructType([StructField("date", StringType(), True)])
dates_df = spark.createDataFrame(data, schema=schema2)

In [11]:
df

,idx,PUMMOK,PUMJONG,UUN,DDD,PPRICE,SSANGI,CORP_NM,ADJ_DT,item,bubin
0,1,감귤,감귤 기타,1.5kg,특(1등),17000,제주자치도 제주시,서울청과,20230407,감귤,11000101
1,2,감귤,감귤 기타,1.5kg,특(1등),17000,제주자치도 제주시,서울청과,20230407,감귤,11000101
2,3,감귤,감귤 기타,1.5kg,특(1등),15000,제주자치도 제주시,서울청과,20230407,감귤,11000101
3,4,감귤,금감,10kg,특(1등),48000,제주자치도 서귀포시,서울청과,20230407,감귤,11000101
4,5,감귤,금감,10kg,특(1등),45000,제주자치도 서귀포시,서울청과,20230407,감귤,11000101
...,...,...,...,...,...,...,...,...,...,...,...
26372,665,호박,쥬키니호박,10kg,특(1등),10500,충청남도 논산시,한국청과,20230407,호박,11000105
26373,666,호박,쥬키니호박,10kg,특(1등),10000,충청남도 논산시,한국청과,20230407,호박,11000105
26374,667,호박,쥬키니호박,10kg,특(1등),8000,충청남도 부여군,한국청과,20230407,호박,11000105
26375,668,호박,쥬키니호박,10kg,특(1등),7500,충청남도 부여군,한국청과,20230407,호박,11000105


In [19]:
from pyspark.sql.types import *

schema = StructType([
    StructField("idx", IntegerType(), True),
    StructField("PUMMOK", StringType(), True),
    StructField("PUMJONG", StringType(), True),
    StructField("UUN", StringType(), True),
    StructField("DDD", StringType(), True),
    StructField("PPRICE", StringType(), True),
    StructField("SSANGI", StringType(), True),
    StructField("CORP_NM", StringType(), True),
    StructField("ADJ_DT", StringType(), True),
    StructField("item", StringType(), True),
    StructField("bubin", StringType(), True),
])

In [26]:
spark_df = spark.createDataFrame(df, schema = schema)

In [27]:
spark_df.show()

Py4JJavaError: An error occurred while calling o148.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChe

In [12]:
spark_df = spark.createDataFrame(transform_flat('20230407'), schema = schema)

AttributeError: 'DataFrame' object has no attribute 'iteritems'

In [8]:
result_df = input_df.withColumn("transformed_data", transform_flat_udf(input_df["date"]))

In [8]:
from pyspark.sql.functions import explode

flattened_result_df = result_df.select("date", explode("transformed_data").alias("flat_data"))

In [10]:
save_path = "C:/Users/wldnr/OneDrive/바탕 화면/DF/Work_JO/CP2/refacto_JO/spark_dataframe"

In [15]:
final_result_df = flattened_result_df.select(
    "date",
    "flat_data.idx",
    "flat_data.PUMMOK",
    "flat_data.PPRICE"
)

final_result_df

DataFrame[date: string, idx: int, PUMMOK: string, PPRICE: string]

In [12]:
final_result_df.printSchema()

root
 |-- date: string (nullable = true)
 |-- idx: integer (nullable = true)
 |-- PUMMOK: string (nullable = true)
 |-- PPRICE: double (nullable = true)



In [13]:
final_result_df.show(1)

Py4JJavaError: An error occurred while calling o70.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:157)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChec

In [24]:
# Define a custom function to square a number
def square(x):
    return x * x

# Register the custom function as a UDF
square_udf = udf(square, IntegerType())

# Create a sample DataFrame
data = [(1,), (2,), (3,), (4,)]
columns = ["number"]
df = spark.createDataFrame(data, columns)

# Use the UDF in a DataFrame operation
result_df = df.select(square_udf(df["number"]).alias("squared_number"))

In [27]:
df

DataFrame[number: bigint]

In [28]:
result_df

DataFrame[squared_number: int]

In [29]:
spark.stop()